# Port Statistics

This notebook develops and explores the various port statistics used in the [Port Performance Project](https://github.com/epistemetrica/Port-Performance-Project). See the README.md file in the main directory for more info.

The primary data set comes from a combination of AIS vessel data and port data, processed in the Port Geodata notebook.

Statistics and final dataframes developed here are used in the Port Performance Dashboard.



In [69]:
#prelims
import polars as pl
import polars.selectors as cs
import pandas as pd
import geopandas as gpd
import time
import plotly.express as px
import matplotlib.pyplot as plt
import contextily as cx
import numpy as np
import glob
import folium
from folium.plugins import HeatMap

#enable string cache for polars categoricals
pl.enable_string_cache()
#display settings
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pl.Config(tbl_rows=100);

## Load Data from geodata_prep notebook

In [70]:
#load data from parquet
main_lf = pl.scan_parquet('port data/dashboard/main.parquet')
#inspect
display(main_lf.describe())
main_lf.limit(5).collect()

statistic,mmsi,time,speed,course,heading,status,vessel_name,vessel_type,imo,length,width,draft,cargo,which_port_waters,port_waters1,port_waters2,port_waters3,status_duration,dock_id,dock_name,facility_type,port_name,vessel_lat,vessel_lon,dock_lat,dock_lon,port_lat,port_lon,call_id
str,str,str,f64,f64,f64,f64,str,f64,f64,f64,f64,f64,f64,str,str,str,str,str,str,str,str,str,f64,f64,f64,f64,f64,f64,str
"""count""","""1437744""","""1437744""",1.437744e6,1.434975e6,1.417856e6,1.437744e6,"""1437744""",1.437744e6,1.437735e6,1.437744e6,1.333436e6,1.329142e6,1.145414e6,"""1437744""","""1437744""","""1437744""","""1437744""","""1434226""","""1437744""","""1437744""","""1433259""","""1437744""",1.437744e6,1.437744e6,1.437744e6,1.437744e6,1.437744e6,1.437744e6,"""1437744"""
"""null_count""","""0""","""0""",0.0,2769.0,19888.0,0.0,"""0""",0.0,9.0,0.0,104308.0,108602.0,292330.0,"""0""","""0""","""0""","""0""","""3518""","""0""","""0""","""4485""","""0""",0.0,0.0,0.0,0.0,0.0,0.0,"""0"""
"""mean""",null,"""2021-06-20 04:07:18.082121""",2.063957,183.841407,186.878868,1.99197,null,73.058067,1.0309e7,207.659064,31.806393,10.726139,73.660093,null,null,null,null,"""12:49:03.023647""",null,null,null,null,32.355621,-96.275208,32.233459,-96.449563,32.232387,-96.450552,null
"""std""",null,null,4.76809,91.832307,103.147354,2.447631,null,4.800045,2.8926e7,57.683308,7.582622,2.913345,8.078969,null,null,null,null,null,null,null,null,null,6.41347,20.009369,6.313581,19.883732,6.31332,19.883235,null
"""min""","""205042000""","""2018-01-01 00:35:19""",0.0,0.0,0.0,0.0,null,70.0,0.0,101.0,0.0,-12.8,0.0,"""Albany Port District, NY""","""Albany Port District, NY""","""Guaynabo, PR""","""Not in port waters""","""0:05:01""","""00XE""","""ADM Corpus Christi Grain Eleva…","""Anchorage""","""Albany Port District, NY""",12.37642,-170.62387,17.936081,-166.53444,17.938939,-166.549916,"""205042000_Port of Long Beach, …"
"""25%""",null,"""2019-10-08 12:32:35""",0.0,122.8,94.0,0.0,null,70.0,9.31524e6,179.0,28.0,8.7,70.0,null,null,null,null,"""0:57:07""",null,null,null,null,29.30768,-118.20893,29.308056,-118.21388,29.31049,-118.2095,null
"""50%""",null,"""2021-08-07 03:51:23""",0.1,183.9,182.0,0.0,null,70.0,9.485801e6,189.0,32.0,10.5,70.0,null,null,null,null,"""1:58:48""",null,null,null,null,30.6573,-93.95583,30.656944,-93.965542,30.69123,-93.96069,null
"""75%""",null,"""2023-02-01 08:01:49""",0.2,252.2,272.0,5.0,null,80.0,9.685425e6,229.0,32.0,12.8,80.0,null,null,null,null,"""4:17:59""",null,null,null,null,36.87423,-80.12157,34.716944,-80.122222,34.71693,-80.117801,null
"""max""","""725019920""","""2024-09-30 23:22:34""",102.3,359.9,359.0,15.0,null,89.0,9.901843e8,667.0,60.0,25.5,191.0,"""Yabucoa, PR""","""Yabucoa, PR""","""Yabucoa, PR""","""Yabucoa, PR""","""2191 days, 16:36:18""","""1JHK""","""YUSEN TERMINALS BERTHS 212-221""","""Tie Off""","""Wilmington, NC""",61.24418,144.67134,61.24306,-66.086926,61.23778,-66.096678,"""725019920_Port Everglades, FL_…"


mmsi,time,speed,course,heading,status,vessel_name,vessel_type,imo,length,width,draft,cargo,which_port_waters,port_waters1,port_waters2,port_waters3,status_duration,dock_id,dock_name,facility_type,port_name,vessel_lat,vessel_lon,dock_lat,dock_lon,port_lat,port_lon,call_id
str,datetime[μs],f64,f64,f64,f64,cat,f64,f64,f64,f64,f64,f64,str,str,str,str,duration[μs],str,str,str,str,f64,f64,f64,f64,f64,f64,str
"""205042000""",2022-12-07 23:20:37,9.1,358.1,0.0,0.0,"""DELOS""",80.0,9.877767e6,336.0,60.0,16.5,80.0,"""Port of Long Beach, CA|Port of…","""Port of Long Beach, CA""","""Port of Los Angeles, CA""","""Not in port waters""",2h 34m 41s,"""0V0U""","""ARCO WESTERN PIPELINE CO BERTH…","""Dock""","""Port of Long Beach, CA""",33.36623,-118.04241,33.757222,-118.21888,33.73957,-118.2095,"""205042000_Port of Long Beach, …"
"""205042000""",2022-12-08 01:55:18,0.3,83.8,328.0,1.0,"""DELOS""",80.0,9.877767e6,336.0,60.0,16.5,80.0,"""Port of Long Beach, CA|Port of…","""Port of Long Beach, CA""","""Port of Los Angeles, CA""","""Not in port waters""",6d 18h 3m 47s,"""0V0U""","""ARCO WESTERN PIPELINE CO BERTH…","""Dock""","""Port of Long Beach, CA""",33.62433,-118.0504,33.757222,-118.21888,33.73957,-118.2095,"""205042000_Port of Long Beach, …"
"""205042000""",2022-12-14 19:59:05,0.9,232.6,228.0,0.0,"""DELOS""",80.0,9.877767e6,336.0,60.0,16.5,80.0,"""Port of Long Beach, CA|Port of…","""Port of Long Beach, CA""","""Port of Los Angeles, CA""","""Not in port waters""",4h 35m 49s,"""0V0U""","""ARCO WESTERN PIPELINE CO BERTH…","""Dock""","""Port of Long Beach, CA""",33.62758,-118.05079,33.757222,-118.21888,33.73957,-118.2095,"""205042000_Port of Long Beach, …"
"""205042000""",2022-12-15 00:34:54,0.0,340.4,340.0,5.0,"""DELOS""",80.0,9.877767e6,336.0,60.0,16.5,80.0,"""Port of Long Beach, CA|Port of…","""Port of Long Beach, CA""","""Port of Los Angeles, CA""","""Not in port waters""",1d 8h 25m 12s,"""0V0U""","""ARCO WESTERN PIPELINE CO BERTH…","""Dock""","""Port of Long Beach, CA""",33.75624,-118.21814,33.757222,-118.21888,33.73957,-118.2095,"""205042000_Port of Long Beach, …"
"""205042000""",2022-12-16 09:00:06,0.2,81.3,339.0,0.0,"""DELOS""",80.0,9.877767e6,336.0,60.0,16.5,80.0,"""Port of Long Beach, CA|Port of…","""Port of Long Beach, CA""","""Port of Los Angeles, CA""","""Not in port waters""",1h 2m 19s,"""0V0U""","""ARCO WESTERN PIPELINE CO BERTH…","""Dock""","""Port of Long Beach, CA""",33.75624,-118.21808,33.757222,-118.21888,33.73957,-118.2095,"""205042000_Port of Long Beach, …"


## Generate Stats

In [71]:
#get stats for each call
calls_df = (
    main_lf
    #ensure sorting by vessel and time
    .sort(['mmsi', 'time'])
    #grouby by call id
    .group_by('call_id')
    .agg(
        #port name
        port_name = pl.first('port_name'),
        #dock name
        dock_name = pl.first('dock_name'),
        #dock_id
        dock_id = pl.first('dock_id'),
        #vessel mmsi
        mmsi = pl.first('mmsi'),
        #vessel imo
        imo = pl.first('imo'),
        #vessel size
        vessel_size = pl.first('length'),
        #time entering port waters
        time_port_entry = pl.col('time').min(),
        #time of arrival at dock
        time_arrival = (
            pl.when(pl.col('status')==5)
            .then(pl.col('time'))
            .otherwise(pl.lit(None))
        ).min(),
        #time of departure from dock
        time_departure = (
            pl.when(pl.col('status')==5)
            .then(pl.col('time')+pl.col('status_duration'))
            .otherwise(pl.lit(None))
        ).max(),
        #time port exit
        time_port_exit = (pl.col('time') + pl.col('status_duration')).max(),
        #hrs a berth
        hrs_at_berth = (
            ((pl.col('status')==5)*
            (pl.col('status_duration').dt.total_minutes()/60)
            )
        ).sum(),
        #hrs at anchor
        hrs_at_anchor = (
            (pl.col('status')==1)*
            (pl.col('status_duration').dt.total_minutes()/60)
        ).sum()
    )
    #drop calls with missing arrival or departure time
    .filter(pl.col('time_arrival').is_not_null() & 
            pl.col('time_departure').is_not_null())
    #drop calls with missing port entry or exit time
    .filter(pl.col('time_port_entry').is_not_null() & 
            pl.col('time_port_exit').is_not_null())
    #compute additional stats
    .with_columns(
        #time from port entry to docking in hrs
        hrs_to_dock = (
            (pl.col('time_arrival') - pl.col('time_port_entry'))
            .dt.total_minutes()/60
        ),
        #time in port waters after leaving dock
        hrs_in_port_after_dock = (
            (pl.col('time_port_exit') - pl.col('time_departure'))
            .dt.total_minutes()/60
        ),
        #total time in port waters in hrs
        hrs_in_port_waters =(
            (pl.col('time_port_exit') - pl.col('time_port_entry'))
            .dt.total_minutes()/60
        )
    )
    #collect
    .collect()
)

#inspect
display(calls_df.describe())
calls_df.head(5)


statistic,call_id,port_name,dock_name,dock_id,mmsi,imo,vessel_size,time_port_entry,time_arrival,time_departure,time_port_exit,hrs_at_berth,hrs_at_anchor,hrs_to_dock,hrs_in_port_after_dock,hrs_in_port_waters
str,str,str,str,str,str,f64,f64,str,str,str,str,f64,f64,f64,f64,f64
"""count""","""148934""","""148934""","""148934""","""148934""","""148934""",148932.0,148934.0,"""148934""","""148934""","""148934""","""148934""",148934.0,148934.0,148934.0,148934.0,148934.0
"""null_count""","""0""","""0""","""0""","""0""","""0""",2.0,0.0,"""0""","""0""","""0""","""0""",0.0,0.0,0.0,0.0,0.0
"""mean""",null,null,null,null,null,1.0159e7,207.545557,"""2021-06-04 17:33:03.693662""","""2021-06-06 06:16:00.676205""","""2021-06-12 20:10:51.460532""","""2021-06-14 16:47:36.207420""",67.771871,14.882013,36.707947,44.604767,239.234188
"""std""",null,null,null,null,null,2.6835e7,58.855356,null,null,null,null,523.532856,178.745156,857.118125,1147.066753,2417.820789
"""min""","""205042000_Port of Long Beach, …","""Albany Port District, NY""","""ADM Corpus Christi Grain Eleva…","""00XE""","""205042000""",0.0,101.0,"""2018-01-01 00:35:19""","""2018-01-01 00:35:19""","""2018-01-01 04:23:54""","""2018-01-01 09:15:57""",0.083333,0.0,0.0,0.0,0.133333
"""25%""",null,null,null,null,null,9.295957e6,176.0,"""2019-08-25 15:08:58""","""2019-08-27 00:57:56""","""2019-09-02 10:44:38""","""2019-09-04 10:01:16""",16.683333,0.0,2.65,2.133333,26.033333
"""50%""",null,null,null,null,null,9.402122e6,190.0,"""2021-07-16 20:53:29""","""2021-07-18 17:52:42""","""2021-07-26 15:04:35""","""2021-07-29 01:04:24""",31.483333,0.0,3.533333,2.783333,44.733333
"""75%""",null,null,null,null,null,9.614294e6,230.0,"""2023-02-23 15:58:06""","""2023-02-25 12:08:11""","""2023-03-04 09:50:11""","""2023-03-06 19:47:43""",58.616667,0.0,5.483333,3.55,85.0
"""max""","""725019920_Port Everglades, FL_…","""Wilmington, NC""","""YUSEN TERMINALS BERTHS 212-221""","""1JHK""","""725019920""",9.901843e8,667.0,"""2024-09-30 12:54:53""","""2024-09-30 16:32:15""","""2024-09-30 23:22:34""","""2024-09-30 23:22:34""",59318.45,52600.6,58913.266667,58950.283333,59112.7


call_id,port_name,dock_name,dock_id,mmsi,imo,vessel_size,time_port_entry,time_arrival,time_departure,time_port_exit,hrs_at_berth,hrs_at_anchor,hrs_to_dock,hrs_in_port_after_dock,hrs_in_port_waters
str,str,str,str,str,f64,f64,datetime[μs],datetime[μs],datetime[μs],datetime[μs],f64,f64,f64,f64,f64
"""477168400_Port of Long Beach, …","""Port of Long Beach, CA""","""CALIFORNIA UNITED TERM BERTH D…","""0VBF""","""477168400""",9.785744e6,366.0,2023-02-24 19:01:18,2023-02-24 23:02:39,2023-03-01 02:19:10,2023-03-01 06:50:20,99.266667,0.0,4.016667,4.516667,107.816667
"""230185000_San Juan, PR_2019-03…","""San Juan, PR""","""MUNICIPAL PIER 15""","""0NAY""","""230185000""",9.002647e6,108.0,2019-03-18 16:42:33,2019-03-18 19:41:22,2019-03-19 05:43:46,2019-03-19 09:45:47,10.033333,0.0,2.966667,4.033333,17.05
"""563000500_PortMiami, FL_2019-0…","""PortMiami, FL""","""PORT OF MIAMI BAYS 144 - 155""","""0QCU""","""563000500""",9.289958e6,335.0,2019-04-27 05:08:50,2019-04-27 09:41:22,2019-04-28 12:12:11,2019-04-28 14:03:22,26.5,0.0,4.533333,1.85,32.9
"""636015248_Port of Kalama, WA_2…","""Port of Kalama, WA""","""EMERALD KALAMA CHEMICAL LLC WH…","""0UX3""","""636015248""",9.528366e6,183.0,2019-04-29 17:49:42,2019-04-29 20:40:33,2019-04-30 12:49:05,2019-04-30 14:51:24,16.133333,0.0,2.833333,2.033333,21.016667
"""538004555_Searsport, ME_2020-0…","""Searsport, ME""","""SEARSPORT MACK POINT MARINE IN…","""0Q7C""","""538004555""",9.373993e6,190.0,2020-02-11 03:07:19,2020-02-11 05:38:21,2020-02-15 11:00:04,2020-02-15 13:03:35,100.95,0.0,2.516667,2.05,105.933333


In [72]:
(calls_df.filter(pl.col('hrs_to_dock')==0).shape[0] /
calls_df.shape[0])

0.019095706823156568

In [73]:
((calls_df.filter((pl.col('hrs_to_dock')==0)&
                (pl.col('hrs_in_port_after_dock')==0)).shape[0]) /
calls_df.filter(pl.col('hrs_to_dock')==0).shape[0])

0.16104078762306612

### Notes on Calls Frame and additional cleaning

- hrs_in_port_after_dock can be 0 when a vessel docks in overlapping port waters and visits both ports. 
    - In such cases, hrs_in_port_waters will be ~ 1/2 that of vessels who only visit 1 port. 
- hrs_to_dock == 0 implies that the first time the vessel sent an AIS message while in port waters was while at dock. This would be expected with new or reset AIS transcievers, and accounts for ~1.9% of port calls. 
    - Of these, ~16% were docked in overlapping port waters prior to visiting the next dock. This would result in hrs_in_port_waters == hrs_at_berth. 
- The mean and quartile statistics for the port calls seems reasonable; however, some calls have very long (6+ years in some cases) hrs_at_berth and related stats. This would result from vessel AIS transponders going offline at some stage during their visit to port waters. Rectifying this issue will be done either in the AIS ingestion or geodata_prep stages at a later date. For now we simply drop these as outliers. 
- Null status_duration values exist in ~3k of 1.3M status changes in the main_df; this is expected whenever an AIS transciever goes offline while in port waters. 
    - There are no observations of entirely-null duration values for mooring statuses, which limits the potential impacts of this issue on the stats.
    - Null status_duration would cause undervalued time data (e.g., time_port_exit == the timestamp of the last status change + the status duration); however, dropping calls with null status_durations has no measurable impact on the statistics, so they are left in the data for now. 

In [74]:
#define zero values drop
def drop_zero_values(df, cols):
    '''
    Drops zero values from the dataframe for the specified columns.
    Args:
        df: Polars DataFrame
        cols: List of columns to drop zero values from
    Returns:
        Polars DataFrame with zero values dropped
    '''
    for col in cols:
        df = df.filter(pl.col(col) > 0)
    return df

#define outlier drop
def drop_outliers(df, cols, threshold=3):
    '''
    Drops outliers from the dataframe for the specified columns.
    Args:
        df: Polars DataFrame
        cols: List of columns to drop outliers from
        threshold: Z-score threshold for outlier detection
    Returns:
        Polars DataFrame with outliers dropped
    '''
    print(f'Outlier threshold: {threshold} Std Devs')
    for col in cols:
        #compute z scores
        df = df.with_columns(
            z_score = (pl.col(col) - pl.col(col).mean()) / pl.col(col).std()
        )
        #drop outliers
        df = df.filter(pl.col('z_score').abs() < threshold)
    return df.drop('z_score')

In [75]:
#get count of rows from calls_df before drop
rows_prior = calls_df.shape[0]

#list cols that should never be zero by construction
zero_cols = ['hrs_at_berth', 'hrs_in_port_waters']
#list cols for outlier drop
outlier_cols = ['hrs_at_berth', 'hrs_to_dock', 'hrs_at_anchor', 
                'hrs_in_port_after_dock', 'hrs_in_port_waters']

#drop zero values
calls_df = drop_zero_values(calls_df, zero_cols)
#print number of observations dropped
print(f'Total zero rows dropped: {rows_prior - calls_df.shape[0]} of {rows_prior}')
rows_prior = calls_df.shape[0]

#drop outliers
#print z_score thresholds
for col in outlier_cols:
    print(f'{col} outlier threshold: {calls_df[col].std()*3/24:.1f} days')
#drop outliers
calls_df = drop_outliers(calls_df, outlier_cols, threshold=3)
#print rows dropped
print(f'Total outlier rows dropped: {rows_prior - calls_df.shape[0]} of {rows_prior}')

#inspect
display(calls_df.describe())
calls_df.head()

Total zero rows dropped: 0 of 148934
hrs_at_berth outlier threshold: 65.4 days
hrs_to_dock outlier threshold: 107.1 days
hrs_at_anchor outlier threshold: 22.3 days
hrs_in_port_after_dock outlier threshold: 143.4 days
hrs_in_port_waters outlier threshold: 302.2 days
Outlier threshold: 3 Std Devs
Total outlier rows dropped: 1411 of 148934


statistic,call_id,port_name,dock_name,dock_id,mmsi,imo,vessel_size,time_port_entry,time_arrival,time_departure,time_port_exit,hrs_at_berth,hrs_at_anchor,hrs_to_dock,hrs_in_port_after_dock,hrs_in_port_waters
str,str,str,str,str,str,f64,f64,str,str,str,str,f64,f64,f64,f64,f64
"""count""","""147523""","""147523""","""147523""","""147523""","""147523""",147522.0,147523.0,"""147523""","""147523""","""147523""","""147523""",147523.0,147523.0,147523.0,147523.0,147523.0
"""null_count""","""0""","""0""","""0""","""0""","""0""",1.0,0.0,"""0""","""0""","""0""","""0""",0.0,0.0,0.0,0.0,0.0
"""mean""",null,null,null,null,null,1.0153e7,207.44387,"""2021-06-07 06:28:46.550544""","""2021-06-07 20:05:58.862075""","""2021-06-10 03:50:15.463093""","""2021-06-10 09:45:14.323122""",49.363115,11.35651,13.612194,5.908665,75.266205
"""std""",null,null,null,null,null,2.6724e7,58.833626,null,null,null,null,71.909922,43.327654,44.041117,46.615158,120.200827
"""min""","""205042000_Port of Long Beach, …","""Albany Port District, NY""","""ADM Corpus Christi Grain Eleva…","""00XE""","""205042000""",0.0,101.0,"""2018-01-01 00:35:19""","""2018-01-01 00:35:19""","""2018-01-01 04:23:54""","""2018-01-01 09:15:57""",0.083333,0.0,0.0,0.0,0.133333
"""25%""",null,null,null,null,null,9.296262e6,176.0,"""2019-08-28 08:02:23""","""2019-08-28 14:16:45""","""2019-08-30 18:18:27""","""2019-08-30 22:20:16""",16.6,0.0,2.65,2.133333,25.9
"""50%""",null,null,null,null,null,9.402043e6,190.0,"""2021-07-19 13:36:33""","""2021-07-20 09:38:57""","""2021-07-22 22:48:02""","""2021-07-23 03:00:20""",31.233333,0.0,3.533333,2.783333,44.283333
"""75%""",null,null,null,null,null,9.61385e6,230.0,"""2023-02-26 06:10:04""","""2023-02-26 17:02:43""","""2023-02-28 22:24:21""","""2023-03-01 02:03:24""",57.866667,0.0,5.45,3.533333,82.866667
"""max""","""725019920_Port Everglades, FL_…","""Wilmington, NC""","""YUSEN TERMINALS BERTHS 212-221""","""1JHK""","""725019920""",9.800025e8,667.0,"""2024-09-30 12:54:53""","""2024-09-30 16:32:15""","""2024-09-30 21:54:12""","""2024-09-30 22:57:53""",1635.516667,530.75,2427.683333,3407.333333,4842.083333


call_id,port_name,dock_name,dock_id,mmsi,imo,vessel_size,time_port_entry,time_arrival,time_departure,time_port_exit,hrs_at_berth,hrs_at_anchor,hrs_to_dock,hrs_in_port_after_dock,hrs_in_port_waters
str,str,str,str,str,f64,f64,datetime[μs],datetime[μs],datetime[μs],datetime[μs],f64,f64,f64,f64,f64
"""477168400_Port of Long Beach, …","""Port of Long Beach, CA""","""CALIFORNIA UNITED TERM BERTH D…","""0VBF""","""477168400""",9.785744e6,366.0,2023-02-24 19:01:18,2023-02-24 23:02:39,2023-03-01 02:19:10,2023-03-01 06:50:20,99.266667,0.0,4.016667,4.516667,107.816667
"""230185000_San Juan, PR_2019-03…","""San Juan, PR""","""MUNICIPAL PIER 15""","""0NAY""","""230185000""",9.002647e6,108.0,2019-03-18 16:42:33,2019-03-18 19:41:22,2019-03-19 05:43:46,2019-03-19 09:45:47,10.033333,0.0,2.966667,4.033333,17.05
"""563000500_PortMiami, FL_2019-0…","""PortMiami, FL""","""PORT OF MIAMI BAYS 144 - 155""","""0QCU""","""563000500""",9.289958e6,335.0,2019-04-27 05:08:50,2019-04-27 09:41:22,2019-04-28 12:12:11,2019-04-28 14:03:22,26.5,0.0,4.533333,1.85,32.9
"""636015248_Port of Kalama, WA_2…","""Port of Kalama, WA""","""EMERALD KALAMA CHEMICAL LLC WH…","""0UX3""","""636015248""",9.528366e6,183.0,2019-04-29 17:49:42,2019-04-29 20:40:33,2019-04-30 12:49:05,2019-04-30 14:51:24,16.133333,0.0,2.833333,2.033333,21.016667
"""538004555_Searsport, ME_2020-0…","""Searsport, ME""","""SEARSPORT MACK POINT MARINE IN…","""0Q7C""","""538004555""",9.373993e6,190.0,2020-02-11 03:07:19,2020-02-11 05:38:21,2020-02-15 11:00:04,2020-02-15 13:03:35,100.95,0.0,2.516667,2.05,105.933333


## Simple delay calculations

Differentiating between delay time and the "efficient" time it takes for a ship to get to a dock is somewhat difficult. 

At its most basic, we can calculate the difference between the hrs_to_dock time for each port call and the minimum hrs_to_dock for that vessel and dock. 

In [76]:
#add min hrs to dock for each vessel-dock pair
delay_df = (
    calls_df
    #min hrs to dock for each vessel-dock pair
    .with_columns(
        min_hrs_to_dock = pl.col('hrs_to_dock').min().over('mmsi', 'dock_id')
    )
    #"delay" in hrs
    .with_columns(
        hrs_delay = pl.col('hrs_to_dock') - pl.col('min_hrs_to_dock')
    )
    #drop unnecessary columns
    .drop('min_hrs_to_dock')
)

delay_df.describe()

statistic,call_id,port_name,dock_name,dock_id,mmsi,imo,vessel_size,time_port_entry,time_arrival,time_departure,time_port_exit,hrs_at_berth,hrs_at_anchor,hrs_to_dock,hrs_in_port_after_dock,hrs_in_port_waters,hrs_delay
str,str,str,str,str,str,f64,f64,str,str,str,str,f64,f64,f64,f64,f64,f64
"""count""","""147523""","""147523""","""147523""","""147523""","""147523""",147522.0,147523.0,"""147523""","""147523""","""147523""","""147523""",147523.0,147523.0,147523.0,147523.0,147523.0,147523.0
"""null_count""","""0""","""0""","""0""","""0""","""0""",1.0,0.0,"""0""","""0""","""0""","""0""",0.0,0.0,0.0,0.0,0.0,0.0
"""mean""",null,null,null,null,null,1.0153e7,207.44387,"""2021-06-07 06:28:46.550544""","""2021-06-07 20:05:58.862075""","""2021-06-10 03:50:15.463093""","""2021-06-10 09:45:14.323122""",49.363115,11.35651,13.612194,5.908665,75.266205,5.250869
"""std""",null,null,null,null,null,2.6724e7,58.833626,null,null,null,null,71.909922,43.327654,44.041117,46.615158,120.200827,27.629323
"""min""","""205042000_Port of Long Beach, …","""Albany Port District, NY""","""ADM Corpus Christi Grain Eleva…","""00XE""","""205042000""",0.0,101.0,"""2018-01-01 00:35:19""","""2018-01-01 00:35:19""","""2018-01-01 04:23:54""","""2018-01-01 09:15:57""",0.083333,0.0,0.0,0.0,0.133333,0.0
"""25%""",null,null,null,null,null,9.296262e6,176.0,"""2019-08-28 08:02:23""","""2019-08-28 14:16:45""","""2019-08-30 18:18:27""","""2019-08-30 22:20:16""",16.6,0.0,2.65,2.133333,25.9,0.0
"""50%""",null,null,null,null,null,9.402043e6,190.0,"""2021-07-19 13:36:33""","""2021-07-20 09:38:57""","""2021-07-22 22:48:02""","""2021-07-23 03:00:20""",31.233333,0.0,3.533333,2.783333,44.283333,0.183333
"""75%""",null,null,null,null,null,9.61385e6,230.0,"""2023-02-26 06:10:04""","""2023-02-26 17:02:43""","""2023-02-28 22:24:21""","""2023-03-01 02:03:24""",57.866667,0.0,5.45,3.533333,82.866667,2.05
"""max""","""725019920_Port Everglades, FL_…","""Wilmington, NC""","""YUSEN TERMINALS BERTHS 212-221""","""1JHK""","""725019920""",9.800025e8,667.0,"""2024-09-30 12:54:53""","""2024-09-30 16:32:15""","""2024-09-30 21:54:12""","""2024-09-30 22:57:53""",1635.516667,530.75,2427.683333,3407.333333,4842.083333,1905.683333


In [77]:
#count of calls with zero delay
delay_df.filter(pl.col('hrs_delay') == 0).shape[0]

66048

64k of 145k port calls align show zero delay indicating those calls represent the only time that that vessel visited that dock. 

### Time Awaiting Berth

We define time awaiting berth as the total time it takes a vessel to get to the dock minus the amount of time the dock was occupied while that vessel was en route. 

In [78]:
%%script echo skipping
#calculate time awaiting berth

#for each call_id and dock, get the total time dock was occupied between time_port_entry and time_arrival

#get time_port_entry, time_arrival for each call id
lf = (calls_df.select('call_id', 'time_port_entry', 'time_arrival')
      .unique().lazy())
#join to main lf
main_lf = main_lf.join(lf, on='call_id', how='left')

for call in calls_df.select('call_id').unique().to_series():
      #get start time and end time
      start = (calls_df.filter(pl.col('call_id')==call)
               .select('time_port_entry').item())
      end = (calls_df.filter(pl.col('call_id')==call)
               .select('time_arrival').item())
      #get dock occupancy
      df = (
            main_lf
            .with_columns(
                  dock_occupied = (
                        (pl.col('status')==5)
                        .then(pl.col('status_duration'))
                        .otherwise(pl.lit(None))
                  )
            )
      )


skipping


In [79]:
#create monthly stats dataframe
monththly_df = (
    calls_df
    #get month from docking time
    .with_columns(
        #extract month from docking time
        month = pl.col('time_arrival').dt.strftime('%Y%m')
    )
    #group by port dock and month
    .group_by(['port_name', 'dock_id', 'month'])
    .agg(
        #count number of vessels
        vessels = pl.n_unique('mmsi'),
        #mean vessel size
        vessel_size_mean = pl.mean('vessel_size'),
        #count number of vessel calls
        calls = pl.n_unique('call_id'),
        #time at dock stats for each vessel in hours
        hrs_occupied = pl.sum('hrs_at_berth'),
        hrs_at_berth_median = pl.median('hrs_at_berth'),
        hrs_at_berth_mean = pl.mean('hrs_at_berth'),
        #time at anchor stats for each vessel visit in hours
        hrs_at_anchor_median = pl.median('hrs_at_anchor'),
        hrs_at_anchor_mean = pl.mean('hrs_at_anchor'),
        #time in port waters 
        hrs_in_port_waters_total = pl.sum('hrs_in_port_waters'),
        hrs_in_port_waters_mean = pl.mean('hrs_in_port_waters'),
        hrs_in_port_waters_median = pl.median('hrs_in_port_waters')
    )
    #get hours from each month
    .with_columns(
        hrs_in_month = (
            pl.when(pl.col('month').str.tail(2).is_in(['01', '03', '05', '07',
                                                       '08', '10', '12']))
            .then(31*24)
            .when(pl.col('month').str.tail(2).is_in(['04', '06', '09', '11']))
            .then(30*24)
            .otherwise(28*24)
        )
    )
    .with_columns(
        #dock utilization - percentage of time a dock is occupied
        utilization = (
            pl.col('hrs_occupied')/pl.col('hrs_in_month')
        )
    )
    #drop hours in month
    .drop('hrs_in_month')
    #sort by port dock then month
    .sort(['port_name', 'dock_id', 'month'])
)

#inspect
display(monththly_df.describe())
monththly_df.head()

statistic,port_name,dock_id,month,vessels,vessel_size_mean,calls,hrs_occupied,hrs_at_berth_median,hrs_at_berth_mean,hrs_at_anchor_median,hrs_at_anchor_mean,hrs_in_port_waters_total,hrs_in_port_waters_mean,hrs_in_port_waters_median,utilization
str,str,str,str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
"""count""","""34533""","""34533""","""34533""",34533.0,34533.0,34533.0,34533.0,34533.0,34533.0,34533.0,34533.0,34533.0,34533.0,34533.0,34533.0
"""null_count""","""0""","""0""","""0""",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"""mean""",null,null,null,3.757305,195.750371,4.271943,210.876402,60.76393,64.510937,9.257232,12.194056,321.532923,93.706709,86.801703,0.288988
"""std""",null,null,null,3.727805,49.119715,4.407909,215.830552,77.042756,78.07731,33.321031,35.128814,379.636602,117.554183,114.401728,0.295455
"""min""","""Albany Port District, NY""","""00XE""","""201801""",1.0,101.0,1.0,0.083333,0.083333,0.083333,0.0,0.0,0.65,0.65,0.65,0.000112
"""25%""",null,null,null,1.0,168.0,1.0,68.316667,22.916667,25.761905,0.0,0.0,98.416667,38.08,34.3,0.093548
"""50%""",null,null,null,3.0,184.8,3.0,149.75,40.916667,44.530556,0.0,0.0,210.0,64.623333,58.366667,0.205085
"""75%""",null,null,null,5.0,211.444444,5.0,284.15,72.825,77.5,0.0,5.027778,402.083333,110.911111,102.2,0.389398
"""max""","""Wilmington, NC""","""1JHK""","""202409""",44.0,385.0,53.0,3430.8,1635.516667,1635.516667,516.55,516.55,6076.233333,4068.6,4068.6,4.61129


port_name,dock_id,month,vessels,vessel_size_mean,calls,hrs_occupied,hrs_at_berth_median,hrs_at_berth_mean,hrs_at_anchor_median,hrs_at_anchor_mean,hrs_in_port_waters_total,hrs_in_port_waters_mean,hrs_in_port_waters_median,utilization
str,str,str,u32,f64,u32,f64,f64,f64,f64,f64,f64,f64,f64,f64
"""Albany Port District, NY""","""0PST""","""201803""",1,171.0,1,67.666667,67.666667,67.666667,0.0,0.0,73.283333,73.283333,73.283333,0.09095
"""Albany Port District, NY""","""0PST""","""201804""",1,189.0,1,165.6,165.6,165.6,0.0,0.0,172.833333,172.833333,172.833333,0.23
"""Albany Port District, NY""","""0PST""","""201806""",1,179.0,1,118.4,118.4,118.4,0.0,0.0,125.083333,125.083333,125.083333,0.164444
"""Albany Port District, NY""","""0PST""","""201807""",2,199.0,2,166.416667,83.208333,83.208333,0.0,0.0,200.683333,100.341667,100.341667,0.223678
"""Albany Port District, NY""","""0PST""","""201808""",3,192.666667,3,185.0,78.45,61.666667,0.0,0.0,350.9,116.966667,98.483333,0.248656


#### Hours calc discussion

The current code first associates the call_id with the month in which the vessel arrived at dock, then counts total times for that call_id to that month. This results in some edge cases where hours stats far exceed the total hours in the month, as in the case that a vessel arrives at the dock and stays there for a very long period of time. 

This can be partially resolved by dropping statuses that are very long, which needs to be done anyway.
- what's the right strategy? set status duration to (the median for that dock? zero? 12hr?) and give an unknown status afterwards? 

It would be fully resolved by totaling monthly hrs (at dock or hrs utilized, for example) independently of call_id.  

In [80]:
#create annual dock stats dataframe
annual_docks_df = (
    calls_df
    #get year from docking time
    .with_columns(
        #extract year from docking time
        year = pl.col('time_arrival').dt.year()
    )
    #group by port dock and month
    .group_by(['port_name', 'dock_id', 'year'])
    .agg(
        #count number of vessels
        vessels = pl.n_unique('mmsi'),
        #mean vessel size
        vessel_size_mean = pl.mean('vessel_size'),
        #count number of vessel calls
        calls = pl.n_unique('call_id'),
        #time at dock stats for each vessel in hours
        hrs_at_berth_median = pl.median('hrs_at_berth'),
        hrs_at_berth_mean = pl.mean('hrs_at_berth'),
        #time at anchor stats for each vessel visit in hours
        hrs_at_anchor_median = pl.median('hrs_at_anchor'),
        hrs_at_anchor_mean = pl.mean('hrs_at_anchor')
    )
    #sort by port then month
    .sort(['port_name', 'year'])
)

#inspect
annual_docks_df.head()

port_name,dock_id,year,vessels,vessel_size_mean,calls,hrs_at_berth_median,hrs_at_berth_mean,hrs_at_anchor_median,hrs_at_anchor_mean
str,str,i32,u32,f64,u32,f64,f64,f64,f64
"""Albany Port District, NY""","""0Y24""",2018,13,167.307692,13,69.3,86.339744,0.0,0.0
"""Albany Port District, NY""","""0RQB""",2018,4,140.0,4,27.591667,27.079167,0.0,0.0
"""Albany Port District, NY""","""0Y0V""",2018,22,148.869565,23,68.0,71.798551,0.0,0.023913
"""Albany Port District, NY""","""0PST""",2018,9,189.333333,9,78.45,85.548148,0.0,0.0
"""Albany Port District, NY""","""0Y0T""",2018,3,131.25,4,63.25,67.55,0.0,0.0


In [81]:
#create annual port stats dataframe
annual_ports_df = (
    calls_df
    #get year from docking time
    .with_columns(
        #extract year from docking time
        year = pl.col('time_arrival').dt.year()
    )
    #group by port dock and month
    .group_by(['port_name', 'year'])
    .agg(
        #count number of vessels
        vessels = pl.n_unique('mmsi'),
        #mean vessel size
        vessel_size_mean = pl.mean('vessel_size'),
        #count number of vessel calls
        calls = pl.n_unique('call_id'),
        #time at dock stats for each vessel in hours
        hrs_at_berth_median = pl.median('hrs_at_berth'),
        hrs_at_berth_mean = pl.mean('hrs_at_berth'),
        #time at anchor stats for each vessel visit in hours
        hrs_at_anchor_median = pl.median('hrs_at_anchor'),
        hrs_at_anchor_mean = pl.mean('hrs_at_anchor')
    )
    #sort by port then month
    .sort(['port_name', 'year'])
)

#inspect
annual_ports_df.head()

port_name,year,vessels,vessel_size_mean,calls,hrs_at_berth_median,hrs_at_berth_mean,hrs_at_anchor_median,hrs_at_anchor_mean
str,i32,u32,f64,u32,f64,f64,f64,f64
"""Albany Port District, NY""",2018,62,154.637681,69,66.85,73.283333,0.0,0.007971
"""Albany Port District, NY""",2019,30,156.909091,33,69.366667,74.917677,0.0,0.018182
"""Albany Port District, NY""",2020,32,147.314286,35,67.55,78.772857,0.0,0.0
"""Albany Port District, NY""",2021,45,157.574468,47,59.8,73.958156,0.0,0.0
"""Albany Port District, NY""",2022,56,160.183333,60,75.741667,83.167222,0.0,1.5075


In [82]:
ports_alltime_df = (
    calls_df
    #group by port 
    .group_by('port_name')
    .agg(
        #count number of vessels
        vessels = pl.n_unique('mmsi'),
        #mean vessel size
        vessel_size_mean = pl.mean('vessel_size'),
        #count number of vessel calls
        calls = pl.n_unique('call_id'),
        #time at dock stats for each vessel in hours
        hrs_at_berth_median = pl.median('hrs_at_berth'),
        hrs_at_berth_mean = pl.mean('hrs_at_berth'),
        #time at anchor stats for each vessel visit in hours
        hrs_at_anchor_median = pl.median('hrs_at_anchor'),
        hrs_at_anchor_mean = pl.mean('hrs_at_anchor')
    )
    #sort by port
    .sort('port_name')
)
#inspect
display(ports_alltime_df.describe())
ports_alltime_df.head()

statistic,port_name,vessels,vessel_size_mean,calls,hrs_at_berth_median,hrs_at_berth_mean,hrs_at_anchor_median,hrs_at_anchor_mean
str,str,f64,f64,f64,f64,f64,f64,f64
"""count""","""70""",70.0,70.0,70.0,70.0,70.0,70.0,70.0
"""null_count""","""0""",0.0,0.0,0.0,0.0,0.0,0.0,0.0
"""mean""",null,723.457143,196.415192,2107.471429,52.663214,68.155777,0.83131,10.308842
"""std""",null,739.279608,36.522594,2255.741002,103.512308,99.767816,2.968721,12.309811
"""min""","""Albany Port District, NY""",3.0,117.744589,3.0,8.233333,10.619192,0.0,0.0
"""25%""",null,129.0,172.540665,409.0,22.733333,36.258571,0.0,1.126932
"""50%""",null,567.0,189.894924,1231.0,34.15,50.47661,0.0,5.548131
"""75%""",null,1007.0,214.206651,3290.0,50.816667,62.83341,0.0,16.349137
"""max""","""Wilmington, NC""",2933.0,295.336091,9817.0,854.516667,797.035897,18.7,52.768775


port_name,vessels,vessel_size_mean,calls,hrs_at_berth_median,hrs_at_berth_mean,hrs_at_anchor_median,hrs_at_anchor_mean
str,u32,f64,u32,f64,f64,f64,f64
"""Albany Port District, NY""",263,157.775676,370,65.733333,79.614955,0.0,0.249459
"""Anacortes, WA""",280,219.134991,1126,37.15,50.601673,0.0,30.749216
"""Baltimore, MD""",2664,213.563386,7549,34.15,49.169592,0.0,16.349137
"""Beaumont, TX""",1018,183.099843,1913,54.016667,69.289214,0.0,4.299599
"""Boston, MA""",319,295.336091,967,17.666667,29.533557,0.0,2.371717


## Visualizations

In [83]:
#make geodataframe
#get port location data
df = main_lf.select('port_name', 'port_lat', 'port_lon').unique().collect()
#convert to geopandas
gdf = gpd.GeoDataFrame(
    ports_alltime_df
    #join to port location data
    .join(df, on='port_name', how='left')
    #convert to pandas dataframe
    .to_pandas(),
    #set geometry
    geometry = gpd.points_from_xy(df['port_lon'], df['port_lat'], crs=4326)
)
gdf.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 70 entries, 0 to 69
Data columns (total 11 columns):
 #   Column                Non-Null Count  Dtype   
---  ------                --------------  -----   
 0   port_name             70 non-null     object  
 1   vessels               70 non-null     uint32  
 2   vessel_size_mean      70 non-null     float64 
 3   calls                 70 non-null     uint32  
 4   hrs_at_berth_median   70 non-null     float64 
 5   hrs_at_berth_mean     70 non-null     float64 
 6   hrs_at_anchor_median  70 non-null     float64 
 7   hrs_at_anchor_mean    70 non-null     float64 
 8   port_lat              70 non-null     float64 
 9   port_lon              70 non-null     float64 
 10  geometry              70 non-null     geometry
dtypes: float64(7), geometry(1), object(1), uint32(2)
memory usage: 5.6+ KB


In [84]:
#scatterplot
fig = px.scatter_geo(
    gdf,
    lon='port_lon',
    lat='port_lat',
    size='vessels',
    color='hrs_at_berth_median',
    range_color=[0,50],
    hover_name='port_name',
    size_max=30,
    title='Total Vessels (all time) and Median Hours at Berth',
    color_continuous_scale=px.colors.sequential.Viridis,
    width=1000,
    height=600,
    labels={
        'time_at_berth_avg':'Hours at Berth'
    }
)

# Fit the view to ports
fig.update_geos(fitbounds="locations")

# Add footnote using add_annotation
fig.add_annotation(
    text="Note: Circle size corresponds to averages vessels per month",  # Footnote text
    xref="paper", yref="paper",  # Position relative to the plot area
    x=0, y=0-0.05,  # Adjust to footnote position
    showarrow=False,  # No arrow, just text
    font=dict(size=14, color="black"),  # Customize the font style
    align="left"
)

# Show the figure
fig.show()

## Point in Time Stats

Still under development

In [85]:
#get point in time stats

#create point in time (pit) df to join stats to
pit_df = (
    main_lf
    .with_columns(
        date = pl.col('time').dt.date(),
        month = pl.col('time').dt.strftime('%Y%m')
    )
    .select('port_name', 'dock_id', 'month', 'date')
    .unique().collect()
)

for hour in range(0, 24):
    #create a time object for each hour
    hour_dt = pl.time(hour)
    #create a dataframe for each hour
    hour_df = (
        main_lf
        .with_columns(
            #get end of status time
            end_time = pl.col('time') + pl.col('status_duration'),
            #get date from time
            date = pl.col('time').dt.date(),
            #get month from time
            month = pl.col('time').dt.strftime('%Y%m')
        )
        #group by port dock and hour
        .group_by(['port_name', 'dock_id', 'month', 'date'])
        .agg(
            #number of vessels at dock at each hour
            vessels_at_dock = (
                #when moored at hour
                pl.when((pl.col('status')==5) & 
                        (hour_dt.is_between(pl.col('time').dt.time(), 
                                         pl.col('end_time').dt.time())))
                #then count the individual vessels
                .then(pl.col('mmsi'))
                .otherwise(pl.lit(None))
                .drop_nulls() #n_unique counts nulls as unique values
                .n_unique()
            ),
            #number of vessels at anchor at each hour
            vessels_at_anchor = (
                #when anchored at hour
                pl.when((pl.col('status')==1) & 
                        (hour_dt.is_between(pl.col('time').dt.time(), 
                                        pl.col('end_time').dt.time())))
                #then count the individual vessels
                .then(pl.col('mmsi'))
                .otherwise(pl.lit(None))
                .drop_nulls()
                .n_unique()
            )
        )
        .collect()
    )
    #join the hour dataframe to the main pit dataframe
    pit_df = (
        pit_df
        .join(hour_df, 
              on=['port_name', 'dock_id', 'month', 'date'], 
              how='left')
        #rename the columns to include the hour
        .rename({
            'vessels_at_dock': f'vessels_at_dock_{hour}',
            'vessels_at_anchor': f'vessels_at_anchor_{hour}'
        })
    )

#get port stats by month
pit_df = (
    pit_df
    #group by port and date
    .group_by(['port_name', 'month', 'date'])
    .agg(
        #sum the number of vessels at all docks at each hour
        cs.starts_with('vessels_at_dock_').sum(),
        #sum the number of vessels at anchor at each hour
        cs.starts_with('vessels_at_anchor_').sum()
    )
    #get the max at any hour
    .with_columns(
        #get max at dock at any hour
        vessels_at_dock_max = (
            pl.max_horizontal(cs.starts_with('vessels_at_dock_'))
        ),
        #get mean at dock any hour
        vessels_at_dock_mean = (
            pl.mean_horizontal(cs.starts_with('vessels_at_dock_'))
        ),
        #get max at anchor at any hour
        vessels_at_anchor_max = (
            pl.max_horizontal(cs.starts_with('vessels_at_anchor_'))
        ),
        #get mean at anchor any hour
        vessels_at_anchor_mean = (
            pl.mean_horizontal(cs.starts_with('vessels_at_anchor_'))
        )
    )
    #select the columns to keep
    .select(['port_name', 'month', 'date', 'vessels_at_dock_max', 
             'vessels_at_dock_mean', 'vessels_at_anchor_max',
             'vessels_at_anchor_mean'])
    #aggregate by month
    .group_by(['port_name', 'month'])
    .agg(
        #get max at anchor on any date during that month
        vessels_at_anchor_max = pl.max('vessels_at_anchor_max'),
        #get mean at anchor on any date during that month
        vessels_at_anchor_mean = pl.mean('vessels_at_anchor_mean'),
        #get max at dock on any date during that month
        vessels_at_dock_max = pl.max('vessels_at_dock_max'),
        #get mean at dock on any date during that month
        vessels_at_dock_mean = pl.mean('vessels_at_dock_mean'
        )
    )
)

In [86]:
#inspect
display(pit_df.describe())
pit_df.head()

statistic,port_name,month,vessels_at_anchor_max,vessels_at_anchor_mean,vessels_at_dock_max,vessels_at_dock_mean
str,str,str,f64,f64,f64,f64
"""count""","""5200""","""5200""",5200.0,5200.0,5200.0,5200.0
"""null_count""","""0""","""0""",0.0,0.0,0.0,0.0
"""mean""",null,null,0.788654,0.047309,2.615769,0.497879
"""std""",null,null,0.803284,0.087055,1.782365,0.478511
"""min""","""Albany Port District, NY""","""201801""",0.0,0.0,0.0,0.0
"""25%""",null,null,0.0,0.0,1.0,0.194444
"""50%""",null,null,1.0,0.014583,2.0,0.350309
"""75%""",null,null,1.0,0.057796,3.0,0.613506
"""max""","""Wilmington, NC""","""202409""",6.0,0.951389,12.0,3.938889


port_name,month,vessels_at_anchor_max,vessels_at_anchor_mean,vessels_at_dock_max,vessels_at_dock_mean
str,str,u32,f64,u32,f64
"""Port of Brunswick, GA""","""202104""",0,0.0,3,0.62931
"""Port of Gulfport, MS""","""202002""",1,0.050926,1,0.152778
"""Greater Lafourche Port, LA""","""201910""",0,0.0,1,0.116667
"""Guaynabo, PR""","""202306""",1,0.020833,3,0.261218
"""Wilmington, NC""","""202103""",0,0.0,4,0.554012


### Max/Mean stats for vessels_at_dock 

- Current output seems far too low - e.g. Port of LA shows a max of 11 vessels at dock at any time since 2018; since there are 37 docks at LA we expect a max in the 20s or higher. 

In [87]:
pit_df.filter(pl.col('port_name')=='Port of Los Angeles, CA').sort('month')

port_name,month,vessels_at_anchor_max,vessels_at_anchor_mean,vessels_at_dock_max,vessels_at_dock_mean
str,str,u32,f64,u32,f64
"""Port of Los Angeles, CA""","""201801""",4,0.348118,7,1.680108
"""Port of Los Angeles, CA""","""201802""",3,0.282738,5,1.077381
"""Port of Los Angeles, CA""","""201803""",2,0.278226,9,1.802419
"""Port of Los Angeles, CA""","""201804""",3,0.3625,7,1.406944
"""Port of Los Angeles, CA""","""201805""",2,0.254032,5,1.443548
"""Port of Los Angeles, CA""","""201806""",2,0.245833,6,1.5375
"""Port of Los Angeles, CA""","""201807""",3,0.280914,6,1.428763
"""Port of Los Angeles, CA""","""201808""",5,0.258065,7,1.608871
"""Port of Los Angeles, CA""","""201809""",3,0.309722,7,1.327778
